# Use xagg to map soil moisture to gage2 boundaries

In [1]:
"""Example notebook to work with nldas soil moisture netcdf files."""

import geopandas as gpd
import numpy as np
import time
import xagg as xa
import pickle
import xarray as xr
import cftime
import datetime
from datetime import timedelta
import rioxarray

In [14]:

ds = xr.open_dataset('../data/bas_ref_all/noan_sm_d_1981.nc')
# this netCDF file contains lat/lon in the attributes.  xagg and indeed cf-standards require lat lon variables os here we add them
ds['x'] = ds.attrs['Lon']
ds['y'] = ds.attrs['Lat']
gdf = gpd.read_file('../data/bas_ref_all/bas_ref_all.shp')

ds

<xarray.Dataset>
Dimensions:          (lat: 224, lon: 464, JD: 365, x: 464, y: 224)
Coordinates:
  * x                (x) float64 -124.9 -124.8 -124.7 ... -67.31 -67.19 -67.06
  * y                (y) float64 25.06 25.19 25.31 25.44 ... 52.69 52.81 52.94
Dimensions without coordinates: lat, lon, JD
Data variables:
    SoilM_0_10cm     (lat, lon, JD) float64 ...
    SoilM_10_40cm    (lat, lon, JD) float64 ...
    SoilM_40_100cm   (lat, lon, JD) float64 ...
    SoilM_100_200cm  (lat, lon, JD) float64 ...
Attributes:
    creation_date:  16-Jul-2021 10:26:34
    Lon:            [-124.9375 -124.8125 -124.6875 -124.5625 -124.4375 -124.3...
    Lat:            [25.0625 25.1875 25.3125 25.4375 25.5625 25.6875 25.8125 ...

In [11]:
ds.rio.set_crs('epsg:4087', inplace=True)
ds.rio.crs

CRS.from_epsg(4087)

In [12]:
ds_alb = ds.rio.reproject('epsg:5071')
ds_alb

MissingSpatialDimensionError: x dimension not found. 'rio.set_spatial_dims()' or using 'rename()' to change the dimension name to 'x' can address this. Data variable: SoilM_0_10cm

In [107]:
gdf.attrs

{}

In [103]:
start = cftime.datetime(1981, 1, 1)
start

cftime.datetime(1981, 1, 1, 0, 0, 0, 0, calendar='gregorian', has_year_zero=False)

In [96]:
times = np.array([start + timedelta(days=d) for d in range(365)])
times[0]

cftime.DatetimeGregorian(1981, 1, 1, 0, 0, 0, 0, has_year_zero=False)

In [97]:
simple_times = np.arange(365)

In [98]:
time_units = 'days since {:%Y-%m-%d 00:00}'.format(times[0])
time_vals = date2num(times, time_units, calendar='gregorian', has_year_zero=False)

In [99]:
soilm_0_10cm = ds['SoilM_0_10cm'].values
# soilm_0_10cm

In [100]:
ds_new = xr.Dataset({'SoilM_0_10cm': (['lat', 'lon', 'time'], soilm_0_10cm, {'units': 'fraction'})}, coords={'lat': (['lat'], ds['lat'].values, {'units':'degrees'}), 'lon': (['lon'], ds['lon'].values, {'units':'degrees'}), 'time': (['time'], times)})

In [101]:
ds_new.time.encoding['units'] = time_units

In [104]:
ds_new.attrs['Conventions'] = 'CF-1.7'
ds_new.attrs['title'] = 'Forecast model run'
ds_new.attrs['nc.institution'] = 'Unidata'
ds_new.attrs['source'] = 'WRF-1.5'
ds_new.attrs['history'] = str(datetime.datetime.utcnow()) + ' Python'
ds_new.attrs['references'] = ''
ds_new.attrs['comment'] = ''

In [105]:
ds_new

<xarray.Dataset>
Dimensions:       (lat: 224, lon: 464, time: 365)
Coordinates:
  * lat           (lat) float64 25.06 25.19 25.31 25.44 ... 52.69 52.81 52.94
  * lon           (lon) float64 -124.9 -124.8 -124.7 ... -67.31 -67.19 -67.06
  * time          (time) object 1981-01-01 00:00:00 ... 1981-12-31 00:00:00
Data variables:
    SoilM_0_10cm  (lat, lon, time) float64 nan nan nan nan ... 37.72 37.73 37.74
Attributes:
    Conventions:     CF-1.7
    title:           Forecast model run
    nc.institution:  Unidata
    source:          WRF-1.5
    history:         2021-09-03 18:23:55.919943 Python
    references:      
    comment:

In [ ]:
ds['crs']= int()
ds.crs.attrs['grid_mapping_name'] = 'latitude_longitude'
ds.crs.attrs['

In [81]:
ds_new.to_netcdf('test2.nc')

In [8]:
start = time.perf_counter()
weightmap = xa.pixel_overlaps(ds,gdf)
end = time.perf_counter()
print(f'finished agg in {round(end-start, 2)} second(s)')

creating polygons for each pixel...
lat/lon bounds not found in dataset; they will be created.
calculating overlaps between pixels and output polygons...
success!
finished agg in 774.12 second(s)


In [9]:
# Save weightmap as pickle file for use later, so we don't have to repeat step above
with open('../data/bas_ref_all/base_ref_all_weights.pkl', 'wb') as file:
    pickle.dump(weightmap, file)

In [10]:
start = time.perf_counter()
aggragated = xa.aggregate(ds, weightmap)
end = time.perf_counter()
print(f'finished agg in {round(end-start, 2)} second(s)')

aggregating SoilM_0_10cm...
aggregating SoilM_10_40cm...
aggregating SoilM_40_100cm...
aggregating SoilM_100_200cm...
all variables aggregated to polygons!
finished agg in 20.75 second(s)


In [11]:
agg_gdf = aggragated.to_dataframe()

In [13]:
# time is represented by suffix 0-365 in variable name
agg_gdf

,AREA,PERIMETER,GAGE_ID,STAID,SoilM_0_10cm0,SoilM_0_10cm1,SoilM_0_10cm2,SoilM_0_10cm3,SoilM_0_10cm4,SoilM_0_10cm5,...,SoilM_100_200cm355,SoilM_100_200cm356,SoilM_100_200cm357,SoilM_100_200cm358,SoilM_100_200cm359,SoilM_100_200cm360,SoilM_100_200cm361,SoilM_100_200cm362,SoilM_100_200cm363,SoilM_100_200cm364
0,2.252690e+09,516060.0,01013500,1013500.0,46.604556,46.604556,46.604556,46.604556,46.604556,46.604556,...,283.046207,282.392907,281.732813,281.069318,280.406537,279.747958,279.094077,278.445109,277.803161,277.168456
1,1.003230e+07,18600.0,01017550,1017550.0,43.899998,43.899998,43.899998,43.899998,43.899998,43.899998,...,304.036774,303.169133,302.299418,301.433873,300.567115,299.695407,298.821653,297.947944,297.078836,296.215178
2,1.610010e+07,27660.0,01021470,1021470.0,45.339730,45.360102,45.349109,45.395380,45.416710,45.419336,...,284.693129,282.955431,281.181461,279.443387,277.774532,276.180272,274.658781,273.210235,271.852628,270.589943
3,7.673850e+07,67980.0,01021480,1021480.0,43.399998,43.385530,43.365554,43.399998,43.399998,43.397019,...,263.468753,261.912001,260.352993,258.833025,257.370488,255.967461,254.621510,253.328986,252.098468,250.926045
4,1.619920e+08,126240.0,01022260,1022260.0,43.833318,43.873973,43.851527,43.893829,43.905254,43.903461,...,266.797224,265.147774,263.488694,261.869914,260.314624,258.825240,257.399197,256.034660,254.742935,253.527677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2052,7.939520e+08,218220.0,06280300,6280300.0,36.093368,36.182638,36.172435,36.156418,36.130549,36.134375,...,319.063083,318.887535,318.715287,318.546230,318.379451,318.216175,318.055891,317.898014,317.744090,317.593378
2053,1.629860e+08,92400.0,06632400,6632400.0,42.440803,42.356329,42.272321,42.300939,42.375772,42.294435,...,230.755850,230.554600,230.364437,230.184390,230.012365,229.850416,229.697504,229.551886,229.415045,229.286383
2054,3.983090e+08,166380.0,09210500,9210500.0,41.124484,41.056848,41.003658,40.939751,40.874247,40.889179,...,249.921609,249.499312,249.087924,248.681821,248.276743,247.876894,247.482913,247.097567,246.720793,246.353916
2055,7.164230e+08,156740.0,10308200,10308200.0,36.479882,36.762226,36.645190,36.614747,37.060082,37.442343,...,229.893001,230.175700,230.079920,229.809979,229.345440,228.716081,227.980385,227.210863,226.623053,226.027838
